In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [2]:
!cp '/content/gdrive/MyDrive/pop_lupus_1.pkl.zip' .

In [3]:
!unzip pop_lupus_1.pkl.zip

Archive:  pop_lupus_1.pkl.zip
  inflating: pop_lupus.pkl           


In [4]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.cluster import KMeans

In [5]:
data = pickle.load(open('pop_lupus.pkl','rb'))
print(data.columns)

Index(['CD45', 'CD196_CCR6', 'CD181_CXCR1', 'HLA_DR', 'CD15', 'CD31_PECAM1',
       'CD8a', 'CD182_CXCR2', 'IgA', 'CD66ace', 'CD63', 'CD14', 'CD66b',
       'CD62L_Lselectin', 'CD3', 'CD27', 'CD86', 'CD10', 'CD197_CCR7', 'CD28',
       'CD11c', 'CD33', 'CD161', 'CD45RO', 'CD24', 'CD38', 'CD278_ICOS',
       'CD32', 'CD152_CTLA4', 'IgM', 'CD184_CXCR4', 'CD279_PD1', 'CD56',
       'CD16', 'Label_0.9', 'score', ' sample type', 'clusters'],
      dtype='object')


In [7]:
populations = {}

for i in set(list(data['clusters'].values)):

    cluster = data.iloc[np.where(data['clusters'].values == i)[0]]
    cluster = cluster.reset_index()
    cluster = cluster.drop(['index'],1)

    if list(cluster['score'].values).count(0) > list(cluster['score'].values).count(1):
        diff = list(cluster['score'].values).count(0)-list(cluster['score'].values).count(1)
        cluster = cluster.drop(list(np.random.choice(np.where(cluster['score'] == 0)[0],diff,replace=False)),0)
    else:
        diff = list(cluster['score'].values).count(1)-list(cluster['score'].values).count(0)
        cluster = cluster.drop(list(np.random.choice(np.where(cluster['score'] == 1)[0],diff,replace=False)),0)
    ##balanced
    populations[i] = cluster 
        
        


In [27]:
keys = populations.keys()
sum = 0
for key in keys:
  print(key, len(populations[key]))
  sum += len(populations[key])
print(sum)

4 53840
9 117340
14 54818
20 184236
152 1040390
156 337140
32 407060
161 285482
35 857758
164 427790
36 300332
42 41098
49 298364
52 344970
59 48038
194 8390300
198 2807128
74 164410
82 371142
91 42504
101 374650
109 118994
17067784


In [15]:
import pandas as pd
import copy as cp
cv = StratifiedKFold(n_splits = 5,shuffle=True)

allvals = []
for key,df in populations.items():

    binary_labels = df['score'].values
    marker_data = df.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1)
    

    for col in marker_data.columns:
        if np.mean(marker_data[col].values) < 0.5:
            marker_data = marker_data.drop([col],1)
  

    if len(marker_data.columns) == 0:
        continue
    cols = marker_data.columns
    save_df = cp.deepcopy(marker_data)
    marker_data = marker_data.values
    result = 0
    all_coef = np.zeros(len(cols))
    for i, (train, test) in enumerate(cv.split(marker_data, binary_labels)):
        X,y = marker_data[train],binary_labels[train]
        X_test,y_test = marker_data[test],binary_labels[test]
        model = LogisticRegression().fit(X,y)
        preds = model.predict(X_test)
        result += accuracy_score(np.round(preds),y_test)
        all_coef+= np.exp(model.coef_[0])
    
    if result/5 > 0.74:
        odds = all_coef/5
        df2 = pd.DataFrame(odds, 
             cols, 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)
        print(key, df2)
        break


            
train_d = save_df[['CD86']].values.reshape(-1,1)
for i, (train, test) in enumerate(cv.split(train_d, binary_labels)):
                X,y = train_d[train],binary_labels[train]
                X_test,y_test = train_d[test],binary_labels[test]
                model = LogisticRegression().fit(X,y)
                preds = model.predict(X_test)
                result += accuracy_score(np.round(preds),y_test)
                print(np.exp(model.coef_[0]))

 

59                  coef
CD11c        2.645795
CD86         2.401396
CD38         2.013826
CD32         1.460879
CD31_PECAM1  1.341582
CD14         1.318181
CD8a         1.045264
HLA_DR       0.845101
CD184_CXCR4  0.675470
CD161        0.581467
CD45         0.486174
CD63         0.482366
CD33         0.378467
[2.24949319]
[2.28052307]
[2.21855217]
[2.25998389]
[2.28355949]


In [9]:
populations[59]

,CD45,CD196_CCR6,CD181_CXCR1,HLA_DR,CD15,CD31_PECAM1,CD8a,CD182_CXCR2,IgA,CD66ace,CD63,CD14,CD66b,CD62L_Lselectin,CD3,CD27,CD86,CD10,CD197_CCR7,CD28,CD11c,CD33,CD161,CD45RO,CD24,CD38,CD278_ICOS,CD32,CD152_CTLA4,IgM,CD184_CXCR4,CD279_PD1,CD56,CD16,Label_0.9,score,sample type,clusters
0,1.948514,1.706604,1.007018,2.097974,1.105328,1.757585,2.680263,1.302616,1.938451,-0.036510,2.594598,3.849777,-3.135161,1.275823,2.041562,0.839154,4.301218,-1.573688,-0.181208,1.095580,2.336453,2.466038,1.416856,1.389178,-0.785995,2.357756,1.077267,2.485542,-0.444082,1.767756,2.977480,-0.783629,4.416359,-2.298224,156,1,NDN,59
1,1.876818,1.749155,1.194624,2.449657,0.948152,1.979195,2.380981,0.814672,2.065706,-0.095658,2.405807,3.855667,-1.712822,1.428125,-0.630650,-0.555127,3.936159,-1.808420,2.319142,-0.831305,2.323361,2.485612,1.928815,1.145741,1.029070,2.535768,-0.469382,2.517945,3.686356,-0.752690,2.961165,0.588757,4.908523,-2.843229,156,1,NDN,59
2,1.540838,-0.788635,0.023576,1.809456,-0.028227,1.884245,0.433583,-0.545685,1.271616,-0.107118,1.588628,3.422979,-2.297421,-1.451333,1.076288,-0.555127,1.286537,0.256276,2.505280,-0.350188,2.001530,2.286413,0.575972,1.554361,-1.767194,1.531289,-0.469382,2.193766,-0.444082,0.417436,1.634627,-0.783629,0.344567,-1.759505,156,1,NDN,59
3,1.609704,-0.769349,0.950817,2.439856,0.577702,1.761012,2.221595,1.793644,2.024448,0.192814,1.645758,2.183045,-2.063813,0.352049,2.732313,-0.555127,4.125047,-1.966389,-0.017905,1.685985,2.410187,1.725270,1.593583,-0.305898,-1.446490,-0.431370,-0.469382,1.552365,-0.444082,1.163333,1.767934,-0.783629,-0.317744,0.744925,58,1,NDN,59
4,-0.103774,-0.788635,-1.164374,0.261743,-1.478926,-1.625656,-0.745235,-1.292081,-1.033252,-2.330029,0.964543,2.008870,-3.135161,2.012260,2.449261,-0.555127,-0.509094,-3.074696,-0.831537,-0.462579,-1.567088,0.392349,0.356468,-0.810028,-1.965733,2.474736,-0.469382,1.302135,-0.444082,0.045144,1.739905,-0.783629,-0.317744,-2.172639,22,1,NDN,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49349,1.734534,2.384005,-0.694510,1.790654,2.249591,1.192137,0.024401,1.685323,1.905497,-0.492346,1.259892,1.678585,-0.596093,1.472123,0.095631,-0.729856,1.702440,-0.549580,1.687921,-0.872675,1.283633,1.581525,0.635239,1.555648,1.968896,1.307056,-0.792294,1.628537,-0.082706,2.232721,1.710858,0.153862,-0.602759,-0.743521,81,0,PBMC,59
49350,1.918225,2.484472,1.895145,1.824782,1.224526,1.571360,1.262172,0.757770,0.836571,-0.492346,1.442355,1.445607,-0.596093,0.710087,-0.367560,-0.729856,1.861738,0.519318,1.590250,-0.531548,1.651785,1.935620,1.579028,0.060474,1.542436,1.136990,-0.792294,1.677516,3.514333,2.504970,1.664401,-0.604048,-0.730720,1.717003,58,0,PBMC,59
49351,1.585608,2.042700,2.090338,1.196758,-0.695733,1.337794,0.032454,1.691901,-0.994015,-0.492346,1.332537,1.496895,1.126140,1.358623,-0.863629,-0.729856,1.688421,2.013801,1.606844,0.758551,1.286333,1.435111,1.135646,1.401838,2.624068,1.768111,1.463420,1.706749,3.634824,2.999611,1.581225,-0.118371,-0.730720,-0.743521,156,0,PBMC,59
49352,1.935562,2.447965,-0.694510,1.879427,1.916928,1.771979,1.343559,0.795692,0.613656,2.069624,1.346129,1.698713,0.323652,1.467546,-0.340795,-0.729856,1.868337,-0.549580,1.666471,0.623466,1.660020,1.956101,1.163752,0.463329,2.015460,1.630195,-0.792294,1.737135,3.573712,2.458741,1.707193,2.196511,1.971825,0.001422,81,0,PBMC,59


In [ ]:
data66 = data.iloc[np.where(data['CD66b'].values > 0.75)]
data66 = data66.reset_index()
data66 = data66.drop(['index'],1)
    
if list(data66[' sample type'].values).count('NDN') > list(data66[' sample type'].values).count('PBMC'):
        diff = list(data66[' sample type'].values).count('NDN') - list(data66[' sample type'].values).count('PBMC')
        data66 = data66.drop(list(np.random.choice(np.where(data66[' sample type'] == 'NDN')[0],diff,replace=False)),0)
else:
        diff = list(data66[' sample type'].values).count('PBMC') - list(data66[' sample type'].values).count('NDN')
        data66 = data66.drop(list(np.random.choice(np.where(data66[' sample type'] == 'PBMC')[0],diff,replace=False)),0)
    ##balanced
binary = data66[' sample type'].values

print(len(data66))

binary_labels = []
for i in binary:
    if i == 'NDN':
        binary_labels.append(0)
    else:
        binary_labels.append(1)
        
binary_labels = np.asarray(binary_labels)

marker_data = data66.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1).values

cv = StratifiedKFold(n_splits = 5,shuffle=True)
result = 0
for i, (train, test) in enumerate(cv.split(marker_data,binary_labels)):
        X,y = marker_data[train],binary_labels[train]
        X_test,y_test = marker_data[test],binary_labels[test]
        model = LogisticRegression().fit(X,y)
        preds = model.predict(X_test)
        result += accuracy_score(np.round(preds),y_test)

print(result/5)
if result/5 > 0.74:
            odds = np.exp(model.coef_[0])
            df2 = pd.DataFrame(odds, 
            data66.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1).columns, 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)
            print(df2)

  

marker_data = data66.drop(['Label_0.9', 'score', ' sample type', 'clusters'],1)['CD66b'].values.reshape(-1,1)

cv = StratifiedKFold(n_splits = 5,shuffle=True)
result = 0
for i, (train, test) in enumerate(cv.split(marker_data,binary_labels)):
        X,y = marker_data[train],binary_labels[train]
        X_test,y_test = marker_data[test],binary_labels[test]
        model = LogisticRegression().fit(X,y)
        preds = model.predict(X_test)
        result += accuracy_score(np.round(preds),y_test)
print(model.coef_[0])


6311090
0.7441590913772422
                     coef
CD66b            7.433718
CD182_CXCR2      1.586399
CD184_CXCR4      1.146967
CD10             1.139034
CD181_CXCR1      1.106686
CD62L_Lselectin  1.092124
IgA              1.073194
CD24             1.057901
CD45RO           1.036478
CD16             1.034144
CD152_CTLA4      1.023113
CD86             1.022427
CD14             0.993772
CD27             0.973413
CD161            0.967858
CD279_PD1        0.967348
CD32             0.964649
CD28             0.956148
IgM              0.945571
CD38             0.939535
CD8a             0.920324
CD15             0.920028
CD278_ICOS       0.913114
HLA_DR           0.897165
CD56             0.892083
CD197_CCR7       0.865917
CD196_CCR6       0.849299
CD3              0.820691
CD31_PECAM1      0.819729
CD66ace          0.806422
CD45             0.769111
CD63             0.756692
CD33             0.714597
CD11c            0.696736
[1.69354301]


In [16]:
print(np.max(populations[59])['CD63'])
print(np.mean(populations[59]['CD86']))
np.std(populations[59]['CD86'])

3.3703455924987793
2.069993495941162


0.7850642204284668

In [22]:
df = populations[59]

df1 = df.iloc[np.where((0.5 <= df['CD63']) & (df['CD63'] < 1.5))]
df2 = df.iloc[np.where((1.5 <= df['CD63']) & (df['CD63'] < 2.5))]
df3 = df.iloc[np.where((2.5 <= df['CD63']) & (df['CD63'] < 3.5))]

prob_stratum_1 = len(df1) / len(df)
prob_stratum_2 = len(df2) / len(df)
prob_stratum_3 = len(df3) / len(df)


df1_small = df1.iloc[np.where((0.5 <= df1['CD86']) & (df1['CD86'] < 2.75))]
df1_big = df1.iloc[np.where((2.75 <= df1['CD86']))]

df2_small = df2.iloc[np.where((0.5 <= df2['CD86']) & (df2['CD86'] < 2.75))]
df2_big = df2.iloc[np.where((2.75 <= df2['CD86']))]

df3_small = df3.iloc[np.where((0.5 <= df3['CD86']) & (df3['CD86'] < 2.75))]
df3_big = df3.iloc[np.where((2.75 <= df3['CD86']))]

# Pr[Y^(0.5 < a < 2.75) = 0] 
probY_a_small = prob_stratum_1 * len(df1_small.iloc[np.where(df1_small['score'] == 0)]) / len(df1_small) 
probY_a_small += prob_stratum_2 * len(df2_small.iloc[np.where(df2_small['score'] == 0)]) / len(df2_small)
probY_a_small += prob_stratum_3 * len(df3_small.iloc[np.where(df3_small['score'] == 0)]) / len(df3_small)

print(probY_a_small)

# Pr[Y^(a >= 2.75) = 0] 
probY_a_big = prob_stratum_1 * len(df1_big.iloc[np.where(df1_big['score'] == 0)]) / len(df1_big)
probY_a_big += prob_stratum_2 * len(df2_big.iloc[np.where(df2_big['score'] == 0)]) / len(df2_big)
probY_a_big += prob_stratum_3 * len(df3_big.iloc[np.where(df3_big['score'] == 0)]) / len(df3_big)

print(probY_a_big)

# Average causal effect of CD86 expressions in range 0.5-2.75 versus CD86 expressions > 2.75
print(probY_a_small - probY_a_big)

0.5633482417027839
0.14624269111414728
0.4171055505886366
